# Import Module&Dataset

In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
import requests
from lmfit import minimize, Parameters, Parameter, report_fit
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')
# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout, ToggleButton, ToggleButtons

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [2]:
response = requests.get('https://api.rootnet.in/covid19-in/stats/history')
print('Request Success? {}'.format(response.status_code == 200))
covid_history = response.json()['data']
keys = ['day', 'total', 'confirmedCasesIndian', 'confirmedCasesForeign', 'confirmedButLocationUnidentified',
        'discharged', 'deaths']
df_covid_history = pd.DataFrame([[d.get('day'), 
                                  d['summary'].get('total'), 
                                  d['summary'].get('confirmedCasesIndian'), 
                                  d['summary'].get('confirmedCasesForeign'),
                                  d['summary'].get('confirmedButLocationUnidentified'),
                                  d['summary'].get('discharged'), 
                                  d['summary'].get('deaths')] 
                                 for d in covid_history],
                    columns=keys)
df_covid_history = df_covid_history.sort_values(by='day')
df_covid_history['infected'] = df_covid_history['total'] - df_covid_history['discharged'] - df_covid_history['deaths']
df_covid_history['total_recovered_or_dead'] = df_covid_history['discharged'] + df_covid_history['deaths']
print(df_covid_history.shape)
df_covid_history.head()

Request Success? True
(458, 9)


,day,total,confirmedCasesIndian,confirmedCasesForeign,confirmedButLocationUnidentified,discharged,deaths,infected,total_recovered_or_dead
0,2020-03-10,47,31,16,0,0,0,47,0
1,2020-03-11,60,44,16,0,0,0,60,0
2,2020-03-12,73,56,17,0,0,0,73,0
3,2020-03-13,82,65,17,0,10,2,70,12
4,2020-03-14,84,67,17,0,10,2,72,12


In [3]:
# def ode_model(z, t, alpha, beta, epsilon, sigma, gamma, mu,nat_d_rate, nat_b_rate):
#     """
#     Reference https://www.idmod.org/docs/hiv/model-seir.html
#     """
#     V, S, E, I, R, D = z
#     N = V + S + E + I + R + D
#     dVdt = alpha*R + epsilon*S - nat_d_rate*V
#     dSdt = -beta*S*I/N-epsilon*S + nat_b_rate*N - nat_d_rate*S
#     dEdt = beta*S*I/N - sigma*E - nat_d_rate*E
#     dIdt = sigma*E - gamma*I - mu*I - nat_d_rate*I
#     dRdt = gamma*I -alpha*R - nat_d_rate*R
#     dDdt = mu*I + nat_d_rate*(N)
#     return [dVdt, dSdt, dEdt, dIdt, dRdt, dDdt]

# Define model

In [31]:
def ode_model(z, t, alpha, beta, epsilon, sigma, gamma, mu, rou, theta, nat_d_rate, nat_b_rate):
    """
    Reference https://www.idmod.org/docs/hiv/model-seir.html
    """
    V, S, E, A, I, R, D = z
    N = V + S + E + I + A + R + D
    dVdt = alpha*R + epsilon*S - nat_d_rate*V
    dSdt = -beta*S*I/N-epsilon*S + nat_b_rate*N - nat_d_rate*S
    dEdt = beta*S*I/N - sigma*E - nat_d_rate*E - rou*E
    dIdt = sigma*E - gamma*I - mu*I - nat_d_rate*I
    dAdt = rou*E-theta*A - nat_d_rate*I
    dRdt = gamma*I -alpha*R - nat_d_rate*R+ theta*A
    dDdt = mu*I + nat_d_rate*N
    return [dVdt, dSdt, dEdt, dAdt, dIdt, dRdt, dDdt]

In [32]:
def ode_solver(t, initial_conditions, params, nat_d_rate, nat_b_rate):
    initV, initE, initI, initA, initR, initN, initD = initial_conditions
    
    alpha = params['alpha'].value
    beta = params['beta'].value
    epsilon = params['epsilon'].value
    sigma = params['sigma'].value
    gamma = params['gamma'].value
    mu = params['mu'].value
    rou = params['rou'].value
    theta = params['theta'].value
    
    initS = initN - (initV + initA + initE + initI + initR + initD)
    res = odeint(ode_model, 
                 [initV, initS, initE, initA, initI, initR, initD], 
                 t, 
                 args=(alpha, beta, epsilon,sigma, gamma, mu, rou, theta, nat_d_rate, nat_b_rate)
                )
    return res

In [33]:
def error(params, initial_conditions, tspan, data, nat_d_rate, nat_b_rate):
    sol = ode_solver(tspan, initial_conditions, params, nat_d_rate, nat_b_rate)
#     print(sol[45,:])
    return (sol[:, 4:7] - data).ravel()

# Parameters

In [196]:
initN = 1380000000
# S0 = 966000000
initV = 0
initE = 3000
initI = 67
initA = 50
initR = 0
initD = 0

alpha = .02
sigma = 1/5.2
gamma = 1/2.9
epsilon = 0.02
rou = 0.05
theta=0.7



mu = 0.034
R0 = 4
beta = R0 * gamma
nat_b_rate= 0.00174
nat_d_rate = 0#.000000073
days = 100

params = Parameters()
params.add('alpha', value=alpha, min=0, max=10)
params.add('beta', value=beta, min=0, max=10)
params.add('sigma', value=sigma, min=0, max=10)
params.add('gamma', value=gamma, min=0, max=10)
params.add('epsilon', value=epsilon, min=0, max=10)
params.add('mu', value=mu, min=0, max=10)
params.add('rou', value=rou, min=0, max=10)
params.add('theta', value=theta, min=0, max=10)


initial_conditions = [initV, initE, initI, initA, initR, initN, initD]
# print(initE,initI)
tspan = np.arange(0, days, 1)
observed_IRD = df_covid_history.loc[:, ['infected', 'discharged', 'deaths']].values
data = df_covid_history.loc[0:(days-1), ['infected', 'discharged', 'deaths']].values

# Training

In [197]:
result = minimize(error, 
                  params, 
                  args=(initial_conditions, tspan, data, nat_d_rate, nat_b_rate), 
                  method='leastsq')
final = data + result.residual.reshape(data.shape)
print(final.shape)

(100, 3)


In [198]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=tspan, y=data[:, 0], mode='markers', name='Observed Infections', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=tspan, y=data[:, 1], mode='markers', name='Observed Recovered', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=tspan, y=data[:, 2], mode='markers', name='Observed Deaths', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=tspan, y=final[:, 0], mode='lines+markers', name='Fitted Infections'))
fig.add_trace(go.Scatter(x=tspan, y=final[:, 1], mode='lines+markers', name='Fitted Recovered'))
fig.add_trace(go.Scatter(x=tspan, y=final[:, 2], mode='lines+markers', name='Fitted Deaths'))
fig.update_layout(title='VSEIARD: Observed vs Fitted',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                       width=1000, 
                       height=600
                     )

# Validating

In [199]:
print('beta:{}'.format(result.params['beta'].value))
print('sigma:{}'.format(result.params['sigma'].value))
print('gamma:{}'.format(result.params['gamma'].value))
print('mu:{}'.format(result.params['mu'].value))
print('alpha:{}'.format(result.params['alpha'].value))
print('epsilon:{}'.format(result.params['epsilon'].value))
print('rou:{}'.format(result.params['rou'].value))
print('theta:{}'.format(result.params['theta'].value))

beta:2.1268523753903255
sigma:0.07404933590231477
gamma:0.020354420508338356
mu:0.002694622478426334
alpha:1.54458218326603e-06
epsilon:0.013400787347854215
rou:0.8785381162620254
theta:0.0014300221597107754


In [200]:
# print(params, nat_d_rate, nat_b_rate)
params['beta'].value = result.params['beta'].value
params['sigma'].value = result.params['sigma'].value
params['gamma'].value = result.params['gamma'].value
params['mu'].value = result.params['mu'].value
params['alpha'].value = result.params['alpha'].value
params['epsilon'].value = result.params['epsilon'].value
params['rou'].value = result.params['rou'].value
params['theta'].value = result.params['theta'].value
# print(params, nat_d_rate, nat_b_rate)

val_days = 450
tspan = np.arange(0, val_days, 1)
fitted_predicted = ode_solver(tspan, initial_conditions, params, nat_d_rate, nat_b_rate)
fitted_predicted_IRD = fitted_predicted[:, 3:7]

In [201]:
# val_days = 200
# tspan = np.arange(0, val_days, 1)
# observed_IRD = df_covid_history.loc[:, ['infected', 'discharged', 'deaths']].values
data = df_covid_history.loc[0:(val_days-1), ['infected', 'discharged', 'deaths']].values

fig = go.Figure()
fig.add_trace(go.Scatter(x=tspan, y=data[:, 0], mode='markers', name='Observed Infections', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=tspan, y=data[:, 1], mode='markers', name='Observed Recovered', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=tspan, y=data[:, 2], mode='markers', name='Observed Deaths', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=tspan, y=fitted_predicted_IRD[:, 0]+fitted_predicted_IRD[:, 1], mode='lines+markers', name='Fitted Infections'))
fig.add_trace(go.Scatter(x=tspan, y=fitted_predicted_IRD[:, 2], mode='lines+markers', name='Fitted Recovered'))
fig.add_trace(go.Scatter(x=tspan, y=fitted_predicted_IRD[:, 3], mode='lines+markers', name='Fitted Deaths'))
fig.update_layout(title='VSEIARD: Observed vs Fitted（{} Days)'.format(val_days),
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=1000, height=600
                     )

# Parameter search

In [71]:

def run(initV, initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days,observed_IRD,data):
    initial_conditions = [initV, initE, initI, initR, initN, initD]
    params['beta'].value, params['sigma'].value,params['gamma'].value, params['mu'].value = [beta, sigma, gamma, mu]
    tspan = np.arange(0, days, 1)
    
#     print(type(error))
    result = minimize(error, params, args=(initial_conditions, tspan, data), method='leastsq')
    
    tspan_fit_pred = np.arange(0, observed_IRD.shape[0], 1)
    params['beta'].value = result.params['beta'].value
    params['sigma'].value = result.params['sigma'].value
    params['gamma'].value = result.params['gamma'].value
    params['mu'].value = result.params['mu'].value
    tspan_fit_pred = np.arange(0, observed_IRD.shape[0], 1)

    sol = ode_solver(tspan_fit_pred, initial_conditions, params)
#     print(sol.shape)
    S, E, I, R, D = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4]
#     print(I.shape,observed_IRD.shape[0])
    return np.mean(np.abs(I[:days] - observed_IRD[:days, 0]))

# run(initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days,observed_IRD,data)
errorlist=[]
# for initE in range(10,100,100):
#     for initI in range(1,100,10):
initN = 1380000000
# S0 = 966000000
initV = 0
initE = 1000
initI = 47
initR = 0
initD = 0

alpha = 0.02
sigma = 1/5.2
gamma = 1/2.9
epsilon = 0.02
mu = 0.034
R0 = 4
beta = R0 * gamma
nat_b_rate=0.0174
nat_d_rate = 0.0073
days = 50

initial_conditions = [initE, initI, initR, initN, initD]
# print(initE,initI)
observed_IRD = df_covid_history.loc[:, ['infected', 'discharged', 'deaths']].values
data = df_covid_history.loc[0:(days-1), ['infected', 'discharged', 'deaths']].values
# e = run( initV, initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days,observed_IRD,data)
# errorlist.append((e,initE,initI))

In [72]:
# print(errorlist)
min_e = min(errorlist)
opt_E = min_e[1]
opt_I = min_e[2]
print(min_e [0], opt_E,opt_I)

5881.810225856797 1000 47


In [73]:
errorlist

[(5881.810225856797, 1000, 47)]